# Universidad del Valle de Guatemala
## Security Data Science - 10
* Jose Abraham Gutierrez Corado - 19111
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 1: Detección de Pishing

### Importación de Librearias

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
#from yellowbrick.features import Rank2D
#from yellowbrick.features import RadViz

### Explorar Datos

In [2]:
df = pd.read_csv('dataset_pishing.csv')
df.head(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [3]:
df[df.status == 'legitimate'].sample(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
3382,http://www.wisegeek.com/what-is-a-memory-card.htm,0,1,0,0,0,0,0,0,0,...,1,0,0,75,6134,28418,0,0,5,legitimate
5064,https://cummins.jobs/,0,0,0,0,0,0,0,0,0,...,1,1,0,261,-1,828180,0,0,2,legitimate
8043,https://www.iplocation.net/subnet-calculator,0,1,0,0,0,0,0,1,0,...,1,0,0,1479,5095,11056,0,0,5,legitimate
7824,https://www.deerberg.de/de/,0,1,0,0,0,0,0,1,0,...,0,0,0,0,-1,494382,0,0,4,legitimate
2514,https://support.apple.com/kb/HT204790,0,0,0,0,0,0,0,0,0,...,0,0,0,210,12209,50,0,1,8,legitimate


In [4]:
df[df.status == 'phishing'].sample(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
3613,http://prakasaws.com/templates/beez5/html/com_...,0,0,1,0,0,0,0,1,0,...,1,0,0,287,1905,0,0,1,0,phishing
2464,http://www.support-appleld.com.secureupdate.du...,1,1,1,0,0,0,0,0,1,...,1,1,0,25,3993,5707171,0,1,0,phishing
2134,https://admin0722.typeform.com/to/Zwa4zP,0,0,0,0,0,0,0,0,0,...,0,0,0,333,6606,899,0,1,5,phishing
2598,http://heckproductions.com/assets/login-user/W...,0,0,0,0,0,0,0,0,0,...,1,0,0,147,4966,0,0,1,1,phishing
9490,http://outdoorism.com.au/includes/customer/e90...,1,0,0,0,0,0,0,0,0,...,1,0,1,0,-1,8661090,0,1,1,phishing


Si se encuentra balanceado.

In [5]:
df.shape

(11430, 67)

### Derivación de Características
Las características del articulo son:
* f1: Full URL length
* f2: Hostname length
* f4-f20: Special Characters in URL
* f25: HTTPS Token
* f26: Ratio of digits in full URLs
* f27: Ratio of digits in hostnames

In [16]:
# Creamos un nuevo dataframe para que sea más fácil la manipulación de datos
dfNew = df[['url', 'status']]
dfNew.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


### Listado de Partes de una URL
* Protocolo HTTPS
* Subdominio
* Dominio
* TLD
* Ruta
* Parámetro
* Etiqueta

### Funciones

In [17]:
# Obtenemos F1
dfNew['length'] = dfNew['url'].str.len()

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_8308\3796037450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['length'] = dfNew['url'].str.len()


In [30]:
#Obtenemos F2
from urllib.parse import urlparse

def getHostname(dfNew):
    hostnames = []
    for url in dfNew.url:
        hostnames.append('{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(url)))
    
    dfNew['hostname_length'] = hostnames
    
    dfNew['hostname_length'] = dfNew['hostname_length'].str.len()
    
    return dfNew

#Obtenemos la columna de las carateristicas F4 - F20
def SpecialCharacters(dfNew):
    sChars = [".","-", "@", "?", "&", "|", "=", "_", "~", "%", "/", "*", ":", ",", ";", "$", "%20", " "]
    cont = 0
    scCount = []
    for url in dfNew.url:
        for c in url:
            if (c in sChars) and (c != "%"):
                cont += 1
                
            elif c == "%":
                c2 = url.index(c)
                c2 = c + url[c2]
                
                if c2 == "%2":
                    cont += 1
                else:
                    cont += 1
                    
        scCount.append(cont)
        cont = 0
        
    dfNew['Special_Char_Count'] = scCount
    
    return dfNew

#Obtenemos si la URL tiene HTTPS o no
def Check4HTTPS(dfNew):
    check = []
    for url in dfNew.url:
        if "https" in url:
            check.append(1)
        else:
            check.append(0)
    
    dfNew['HTTPS'] = check
    return dfNew

#Obtenemos el ratio de números en la URL y en el Hostname
def ratioDigits(dfNew):
    digits = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    hostnames = []
    urlDigits = []
    hostnameDigits = []
    cont1 = 0
    cont2 = 0
    
    for url in dfNew.url:
        for c in url:
            if c in digits:
                cont1 += 1
                
        urlDigits.append(cont1/len(url))
        cont1 = 0
        hostnames.append('{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(url)))
    
    for hn in hostnames:
        for c in hn:
            if c in digits:
                cont2 += 1
                
        hostnameDigits.append(cont2/len(hn))
        cont2 = 0
        
    dfNew['urlDigits_ratio'] = urlDigits
    dfNew['hostnameDigits_ratio'] = hostnameDigits
    
    return dfNew

In [31]:
dfNew = getHostname(dfNew)
dfNew = SpecialCharacters(dfNew)
dfNew = Check4HTTPS(dfNew)
dfNew = ratioDigits(dfNew)
dfNew

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_8308\1161462486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['hostname_length'] = hostnames
C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_8308\1161462486.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['hostname_length'] = dfNew['hostname_length'].str.len()
C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_8308\1161462486.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,url,status,length,hostname_length,Special_Char_Count,HTTPS,urlDigits_ratio,hostnameDigits_ratio
0,http://www.crestonwood.com/router.php,legitimate,37,27,7,0,0.000000,0.0
1,http://shadetreetechnology.com/V4/validation/a...,phishing,77,31,7,0,0.220779,0.0
2,https://support-appleld.com.secureupdate.duila...,phishing,126,59,19,1,0.150794,0.0
3,http://rgipt.ac.in,legitimate,18,19,5,0,0.000000,0.0
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate,55,23,10,0,0.000000,0.0
...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,legitimate,45,25,7,0,0.000000,0.0
11426,http://www.budgetbots.com/server.php/Server%20...,phishing,84,26,15,0,0.023810,0.0
11427,https://www.facebook.com/Interactive-Televisio...,legitimate,105,25,17,1,0.142857,0.0
11428,http://www.mypublicdomainpictures.com/,legitimate,38,38,6,0,0.000000,0.0


In [32]:
dfNew.sample(5)

,url,status,length,hostname_length,Special_Char_Count,HTTPS,urlDigits_ratio,hostnameDigits_ratio
1126,https://www.checkpoint.com/download/public-fil...,legitimate,72,27,11,1,0.000000,0.0
9810,http://www.siliguribaghajatinsportingclub.com/...,phishing,118,46,11,0,0.194915,0.0
1193,http://justlookapp.com/blog/wp-content/themes/...,phishing,103,23,13,0,0.155340,0.0
10468,http://www.dell.com/support/article/bz/en/bzdh...,legitimate,58,20,12,0,0.120690,0.0
31,http://mno.naomigoff.com/Apollo/bc28a52c2070ca...,phishing,64,25,7,0,0.343750,0.0
